## MIGUEL

In [41]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [42]:
df = pd.read_csv('../df_cleaned.csv')

In [43]:
df.drop(columns='Unnamed: 0', inplace=True)

In [44]:
df

,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,0,51,1,1,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,1,54,1,3,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,2,63,1,4,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,3,52,0,2,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,4,55,1,4,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,179,54,1,4,200.0,198.0,0.0,0.0,142.0,1.0,2.0,2.0,NaN,NaN,NaN
912,180,55,1,2,110.0,214.0,1.0,1.0,180.0,0.0,NaN,NaN,NaN,NaN,NaN
913,181,67,1,3,152.0,212.0,0.0,2.0,150.0,0.0,0.8,2.0,0.0,7.0,NaN
914,182,59,1,1,170.0,288.0,0.0,2.0,159.0,0.0,0.2,2.0,0.0,7.0,NaN


In [45]:
df.drop('index', axis=1, inplace=True)

In [46]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,51,1,1,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,54,1,3,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,63,1,4,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,52,0,2,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,55,1,4,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,54,1,4,200.0,198.0,0.0,0.0,142.0,1.0,2.0,2.0,NaN,NaN,NaN
912,55,1,2,110.0,214.0,1.0,1.0,180.0,0.0,NaN,NaN,NaN,NaN,NaN
913,67,1,3,152.0,212.0,0.0,2.0,150.0,0.0,0.8,2.0,0.0,7.0,NaN
914,59,1,1,170.0,288.0,0.0,2.0,159.0,0.0,0.2,2.0,0.0,7.0,NaN


In [47]:
df.replace({'<NA>':np.nan}, inplace=True)

In [48]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,51,1,1,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,54,1,3,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,63,1,4,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,52,0,2,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,55,1,4,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,54,1,4,200.0,198.0,0.0,0.0,142.0,1.0,2.0,2.0,NaN,NaN,NaN
912,55,1,2,110.0,214.0,1.0,1.0,180.0,0.0,NaN,NaN,NaN,NaN,NaN
913,67,1,3,152.0,212.0,0.0,2.0,150.0,0.0,0.8,2.0,0.0,7.0,NaN
914,59,1,1,170.0,288.0,0.0,2.0,159.0,0.0,0.2,2.0,0.0,7.0,NaN


In [49]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
label       float64
dtype: object

In [50]:
numeric_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
# Convertir columnas numéricas de nuevo a tipo numérico
df[numeric_cols] = df[numeric_cols].astype(float)

# Convertir columnas categóricas de nuevo a tipo categórico
df[categorical_cols] = df[categorical_cols].astype('category')

# Verificar que los dtypes han cambiado
print(df.dtypes)


age          float64
sex         category
cp          category
trestbps     float64
chol         float64
fbs         category
restecg     category
thalach      float64
exang       category
oldpeak      float64
slope       category
ca          category
thal        category
label        float64
dtype: object


In [51]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,51.0,1,1,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,54.0,1,3,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,63.0,1,4,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,52.0,0,2,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,55.0,1,4,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,54.0,1,4,200.0,198.0,0.0,0.0,142.0,1.0,2.0,2.0,NaN,NaN,NaN
912,55.0,1,2,110.0,214.0,1.0,1.0,180.0,0.0,NaN,NaN,NaN,NaN,NaN
913,67.0,1,3,152.0,212.0,0.0,2.0,150.0,0.0,0.8,2.0,0.0,7.0,NaN
914,59.0,1,1,170.0,288.0,0.0,2.0,159.0,0.0,0.2,2.0,0.0,7.0,NaN


In [52]:
# Convertir '<NA>' a NaN
df.replace('<NA>', np.nan, inplace=True)

# Obtener columnas numéricas y categóricas
numeric_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'label']

# Imputación para variables numéricas (usando la media)
numeric_imputer = SimpleImputer(strategy='mean')  # Cambio a 'mean' en lugar de 'median'
df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

# Imputación para variables categóricas (usando la moda)
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

# Verificar que ya no hay valores faltantes
print(df.isnull().sum())

# El DataFrame 'df' ahora contiene los valores imputados



age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
label       0
dtype: int64


In [53]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,51.0,1.0,1.0,125.000000,213.000000,0.0,2.0,125.000000,1.0,1.400000,1.0,1.0,3.0,0.0
1,54.0,1.0,3.0,120.000000,237.000000,0.0,0.0,150.000000,1.0,1.500000,2.0,0.0,7.0,2.0
2,63.0,1.0,4.0,140.000000,0.000000,0.0,2.0,149.000000,0.0,2.000000,1.0,0.0,3.0,2.0
3,52.0,0.0,2.0,140.000000,199.146727,0.0,0.0,140.000000,0.0,0.000000,2.0,0.0,3.0,0.0
4,55.0,1.0,4.0,140.000000,217.000000,0.0,0.0,111.000000,1.0,5.600000,3.0,0.0,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,54.0,1.0,4.0,200.000000,198.000000,0.0,0.0,142.000000,1.0,2.000000,2.0,0.0,3.0,0.0
912,55.0,1.0,2.0,110.000000,214.000000,1.0,1.0,180.000000,0.0,0.875878,2.0,0.0,3.0,0.0
913,67.0,1.0,3.0,152.000000,212.000000,0.0,2.0,150.000000,0.0,0.800000,2.0,0.0,7.0,0.0
914,59.0,1.0,1.0,170.000000,288.000000,0.0,2.0,159.000000,0.0,0.200000,2.0,0.0,7.0,0.0


In [54]:
train_end = df[0:732]
test_end = df[(916-184):]

In [55]:
test_end

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
732,57.0,1.0,4.0,156.000000,173.000000,0.0,2.0,119.000000,1.0,3.000000,3.0,0.0,3.0,0.0
733,52.0,1.0,2.0,160.000000,196.000000,0.0,0.0,165.000000,0.0,0.000000,2.0,0.0,3.0,0.0
734,48.0,1.0,2.0,100.000000,199.146727,0.0,0.0,100.000000,0.0,0.000000,2.0,0.0,3.0,0.0
735,62.0,1.0,4.0,115.000000,0.000000,0.0,0.0,128.000000,1.0,2.500000,3.0,0.0,3.0,0.0
736,51.0,1.0,3.0,110.000000,175.000000,0.0,0.0,123.000000,0.0,0.600000,1.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,54.0,1.0,4.0,200.000000,198.000000,0.0,0.0,142.000000,1.0,2.000000,2.0,0.0,3.0,0.0
912,55.0,1.0,2.0,110.000000,214.000000,1.0,1.0,180.000000,0.0,0.875878,2.0,0.0,3.0,0.0
913,67.0,1.0,3.0,152.000000,212.000000,0.0,2.0,150.000000,0.0,0.800000,2.0,0.0,7.0,0.0
914,59.0,1.0,1.0,170.000000,288.000000,0.0,2.0,159.000000,0.0,0.200000,2.0,0.0,7.0,0.0


## 4 TRY - Random Forest

In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Identificar la variable objetivo y las características para el entrenamiento
target_column = 'label'  # Nombre real de la variable objetivo
features_train = train_end.drop(columns=[target_column])
target_train = train_end[target_column]

# Identificar las características para la predicción en el conjunto de prueba
features_test = test_end.drop(columns=[target_column])

# Inicializar y entrenar el modelo de Random Forest para clasificación
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(features_train, target_train)

# Predecir las clases de las variables categóricas faltantes en el conjunto de prueba
predicted_classes = model.predict(features_test)

# Crear un nuevo DataFrame con las clases predichas
predicted_df = test_end.copy()
predicted_df[target_column] = predicted_classes

# Imprimir el DataFrame con las clases predichas
print(predicted_df)


      age  sex   cp    trestbps        chol  fbs  restecg     thalach  exang  \
732  57.0  1.0  4.0  156.000000  173.000000  0.0      2.0  119.000000    1.0   
733  52.0  1.0  2.0  160.000000  196.000000  0.0      0.0  165.000000    0.0   
734  48.0  1.0  2.0  100.000000  199.146727  0.0      0.0  100.000000    0.0   
735  62.0  1.0  4.0  115.000000    0.000000  0.0      0.0  128.000000    1.0   
736  51.0  1.0  3.0  110.000000  175.000000  0.0      0.0  123.000000    0.0   
..    ...  ...  ...         ...         ...  ...      ...         ...    ...   
911  54.0  1.0  4.0  200.000000  198.000000  0.0      0.0  142.000000    1.0   
912  55.0  1.0  2.0  110.000000  214.000000  1.0      1.0  180.000000    0.0   
913  67.0  1.0  3.0  152.000000  212.000000  0.0      2.0  150.000000    0.0   
914  59.0  1.0  1.0  170.000000  288.000000  0.0      2.0  159.000000    0.0   
915  58.0  1.0  4.0  132.142357  203.000000  1.0      0.0  137.533101    0.0   

      oldpeak  slope   ca  thal  label 

In [57]:
predicted_df['index'] = predicted_df.index

In [58]:
predicted_df.reset_index(inplace=True)

In [59]:
predicted_df

,level_0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label,index
0,732,57.0,1.0,4.0,156.000000,173.000000,0.0,2.0,119.000000,1.0,3.000000,3.0,0.0,3.0,2.0,732
1,733,52.0,1.0,2.0,160.000000,196.000000,0.0,0.0,165.000000,0.0,0.000000,2.0,0.0,3.0,0.0,733
2,734,48.0,1.0,2.0,100.000000,199.146727,0.0,0.0,100.000000,0.0,0.000000,2.0,0.0,3.0,0.0,734
3,735,62.0,1.0,4.0,115.000000,0.000000,0.0,0.0,128.000000,1.0,2.500000,3.0,0.0,3.0,1.0,735
4,736,51.0,1.0,3.0,110.000000,175.000000,0.0,0.0,123.000000,0.0,0.600000,1.0,0.0,3.0,0.0,736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,911,54.0,1.0,4.0,200.000000,198.000000,0.0,0.0,142.000000,1.0,2.000000,2.0,0.0,3.0,3.0,911
180,912,55.0,1.0,2.0,110.000000,214.000000,1.0,1.0,180.000000,0.0,0.875878,2.0,0.0,3.0,0.0,912
181,913,67.0,1.0,3.0,152.000000,212.000000,0.0,2.0,150.000000,0.0,0.800000,2.0,0.0,7.0,0.0,913
182,914,59.0,1.0,1.0,170.000000,288.000000,0.0,2.0,159.000000,0.0,0.200000,2.0,0.0,7.0,0.0,914


In [60]:
predicted_df.drop(columns=['index'], inplace=True)

# Renombrar la columna 'level_0' a 'index'
predicted_df.rename(columns={'level_0': 'index'}, inplace=True)

predicted_df['ID'] = range(len(predicted_df))
# Guardar las columnas 'index' y 'label' en un archivo CSV
predicted_df[['ID', 'label']].to_csv('try1_data.csv', index=False)

In [61]:
predicted_df

,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label,ID
0,732,57.0,1.0,4.0,156.000000,173.000000,0.0,2.0,119.000000,1.0,3.000000,3.0,0.0,3.0,2.0,0
1,733,52.0,1.0,2.0,160.000000,196.000000,0.0,0.0,165.000000,0.0,0.000000,2.0,0.0,3.0,0.0,1
2,734,48.0,1.0,2.0,100.000000,199.146727,0.0,0.0,100.000000,0.0,0.000000,2.0,0.0,3.0,0.0,2
3,735,62.0,1.0,4.0,115.000000,0.000000,0.0,0.0,128.000000,1.0,2.500000,3.0,0.0,3.0,1.0,3
4,736,51.0,1.0,3.0,110.000000,175.000000,0.0,0.0,123.000000,0.0,0.600000,1.0,0.0,3.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,911,54.0,1.0,4.0,200.000000,198.000000,0.0,0.0,142.000000,1.0,2.000000,2.0,0.0,3.0,3.0,179
180,912,55.0,1.0,2.0,110.000000,214.000000,1.0,1.0,180.000000,0.0,0.875878,2.0,0.0,3.0,0.0,180
181,913,67.0,1.0,3.0,152.000000,212.000000,0.0,2.0,150.000000,0.0,0.800000,2.0,0.0,7.0,0.0,181
182,914,59.0,1.0,1.0,170.000000,288.000000,0.0,2.0,159.000000,0.0,0.200000,2.0,0.0,7.0,0.0,182


In [62]:
try4_df = pd.read_csv('try1_data.csv')

In [63]:
try4_df

,ID,label
0,0,2.0
1,1,0.0
2,2,0.0
3,3,1.0
4,4,0.0
...,...,...
179,179,3.0
180,180,0.0
181,181,0.0
182,182,0.0


In [64]:
try4_df['label'].value_counts()

0.0    96
1.0    37
3.0    27
2.0    24
Name: label, dtype: int64

## 5 TRY - SVM

In [65]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Crear un modelo de clasificador SVM
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0, random_state=42))

# Entrenar el modelo SVM
svm_model.fit(features_train, target_train)

# Predecir las clases en el conjunto de prueba
predicted_classes_svm = svm_model.predict(features_test)

# Crear un nuevo DataFrame con las clases predichas por SVM
predicted_df_svm = test_end.copy()
predicted_df_svm[target_column] = predicted_classes_svm

# Imprimir el DataFrame con las clases predichas por SVM
print(predicted_df_svm)


      age  sex   cp    trestbps        chol  fbs  restecg     thalach  exang  \
732  57.0  1.0  4.0  156.000000  173.000000  0.0      2.0  119.000000    1.0   
733  52.0  1.0  2.0  160.000000  196.000000  0.0      0.0  165.000000    0.0   
734  48.0  1.0  2.0  100.000000  199.146727  0.0      0.0  100.000000    0.0   
735  62.0  1.0  4.0  115.000000    0.000000  0.0      0.0  128.000000    1.0   
736  51.0  1.0  3.0  110.000000  175.000000  0.0      0.0  123.000000    0.0   
..    ...  ...  ...         ...         ...  ...      ...         ...    ...   
911  54.0  1.0  4.0  200.000000  198.000000  0.0      0.0  142.000000    1.0   
912  55.0  1.0  2.0  110.000000  214.000000  1.0      1.0  180.000000    0.0   
913  67.0  1.0  3.0  152.000000  212.000000  0.0      2.0  150.000000    0.0   
914  59.0  1.0  1.0  170.000000  288.000000  0.0      2.0  159.000000    0.0   
915  58.0  1.0  4.0  132.142357  203.000000  1.0      0.0  137.533101    0.0   

      oldpeak  slope   ca  thal  label 

In [66]:
predicted_df_svm['index'] = predicted_df.index
predicted_df_svm.reset_index(inplace=True)

In [67]:
predicted_df_svm.drop(columns=['index'], inplace=True)

# Renombrar la columna 'level_0' a 'index'
predicted_df_svm.rename(columns={'level_0': 'index'}, inplace=True)

predicted_df_svm['ID'] = range(len(predicted_df_svm))
# Guardar las columnas 'index' y 'label' en un archivo CSV
predicted_df_svm[['ID', 'label']].to_csv('try2_data.csv', index=False)

In [68]:
try5_df = pd.read_csv('try2_data.csv')

In [69]:
try5_df

,ID,label
0,0,2.0
1,1,0.0
2,2,0.0
3,3,2.0
4,4,0.0
...,...,...
179,179,2.0
180,180,0.0
181,181,0.0
182,182,0.0


In [70]:
try5_df['label'].value_counts()

0.0    101
1.0     44
3.0     25
2.0     14
Name: label, dtype: int64